# Data Processing script for the NSM/SWEML v2.0
This .ipynb script uses python module for retrieving NASA ASO observations, locating nearest SNOTEL sites, connecting SNOTEL obs with ASO obs, and add geospatial features to the ML training/testing/hindcast dataframes.

In [2]:
from ASOget import ASODownload, ASODataProcessing

# Inputs for fetching ASO data for a region
short_name = 'ASO_50M_SWE'
version = '1'
time_start = '2013-04-02T00:00:00Z'
time_end = '2019-07-19T23:59:59Z'
region = 'S_Sierras'
folder_name = f"SWE_Data/{region}"
output_res = 100 #desired spatial resoultion in meters (m)
directory = "SWE_Data"

#Get ASO data
data_tool = ASODownload(short_name, version)
selected_region = data_tool.select_region(region)  # Call select_region on the instance, S Sierras is #2. There may be a need to modify this in order to cover CONUS, create region folders...

print(f"Fetching file URLs in progress for {selected_region} from {time_start} to {time_end}")
url_list = data_tool.cmr_search(time_start, time_end, data_tool.bounding_box)
data_tool.cmr_download(directory, region)

print('Converting .tif to csv')
data_processor = ASODataProcessing()
data_processor.convert_tiff_to_csv(folder_name, output_res, region) #Takes ~3 mins, can it be multithreaded?

#Applying polygon geometries, please be patient, this step can take a few minutes...Converting to GeoDataFrame
input_folder = f"ASO/{output_res}M_SWE_csv/{region}"
metadata_file = f"grid_cells_meta.csv"
output_folder = f"Processed_SWE/{region}"
data_processor.process_folder(input_folder, metadata_file, output_folder) #Takes ~20 minutes. Can this be multithreaded?

Getting ASO data for S_Sierras, other option are:
1. N_Sierras
2. S_Sierras
3. Greater_Yellowstone
4. N_Co_Rockies
5. SW_Mont
6. SW_Co_Rockies
7. GBasin
8. N_Wasatch
9. N_Cascade
10. S_Wasatch
11. SW_Mtns
12. E_WA_N_Id_W_Mont
13. S_Wyoming
14. SE_Co_Rockies
15. Sawtooth
16. Ca_Coast
17. E_Or
18. N_Yellowstone
19. S_Cascade
20. Wa_Coast
21. Greater_Glacier
22. Or_Coast
Bounding Box: -120.3763448720203,36.29256774541929,-118.292253412863,38.994985247736324
Fetching file URLs in progress for None from 2013-04-02T00:00:00Z to 2019-07-19T23:59:59Z
Querying for data:
	https://cmr.earthdata.nasa.gov/search/granules.json?provider=NSIDC_ECS&sort_key[]=start_date&sort_key[]=producer_granule_id&scroll=true&page_size=2000&short_name=ASO_50M_SWE&version=001&version=01&version=1&temporal[]=2013-04-02T00:00:00Z,2019-07-19T23:59:59Z&bounding_box=-120.3763448720203,36.29256774541929,-118.292253412863,38.994985247736324

Found 131 matches.
['https://n5eil01u.ecs.nsidc.org/DP1/ASO/ASO_50M_SWE.001/2013.04

100%|██████████| 131/131 [03:14<00:00,  1.48s/it]


Applying polygon geometries, please be patient, this step can take a few minutes...
Converting to GeoDataFrame
Processing 99 files for dataframe development


100%|██████████| 99/99 [13:29<00:00,  8.17s/it]


# Code for generating ML dataframe using nearest in situ monitoring sites

In [1]:
import GeoDF

# GeoDF used to create a dataframe for ML model development. Its function is to connect in situ observations to gridded locations
region = 'S_Sierras' #Should be done in above code block
output_res = 100

#load snotel meta location data, use haversive function
GeoDF.fetch_snotel_sites_for_cellids(region) # Check to make sure we are using good sites... run NSM data collection and explore sites that come back as bad

# Get geophysical attributes for each site
gdf = GeoSpatial(region)
GeoDF.extract_terrain_data_threaded(gdf)

#Connect nearest snotel with ASO data, this should be last for now, need to add geophysical characteristics to the site first, then this...
#finaldf = GeoDF.Nearest_Snotel_2_obs(region, output_res, dropna = True) 


Loading all Geospatial prediction/observation files and concatenating into one dataframe


100%|██████████| 99/99 [01:27<00:00,  1.14it/s]


Identifying unique sites to create geophysical information dataframe
Removing nan ASO sites and saving geodataframe to TrainingDFs S_Sierras folder.
converting to geodataframe
Loading SNOTEL metadata and processing snotel geometry
Processing snotel geometry
Calculating haversine distance from each cell to in situ OBS, and saving cell-obs relationships in dictionary


855483it [11:57, 1191.81it/s]


Saving nearest SNOTEL in S_Sierras for each cell id in a pkl file


{'11N_cell_-119.58959364631137_38.186209698720205': ['CDEC:HRS',
  'CDEC:LVT',
  'SNOTEL:575_CA_SNTL',
  'CDEC:LVM',
  'SNOTEL:771_CA_SNTL',
  'CDEC:SPS'],
 '11N_cell_-119.59309813700962_38.184509449472536': ['CDEC:HRS',
  'CDEC:LVT',
  'SNOTEL:575_CA_SNTL',
  'CDEC:LVM',
  'SNOTEL:771_CA_SNTL',
  'CDEC:SPS'],
 '11N_cell_-119.59195797185825_38.18447632413384': ['CDEC:HRS',
  'CDEC:LVT',
  'SNOTEL:575_CA_SNTL',
  'CDEC:LVM',
  'SNOTEL:771_CA_SNTL',
  'CDEC:SPS'],
 '11N_cell_-119.59081780857791_38.184443187748315': ['CDEC:HRS',
  'CDEC:LVT',
  'SNOTEL:575_CA_SNTL',
  'CDEC:LVM',
  'SNOTEL:771_CA_SNTL',
  'CDEC:SPS'],
 '11N_cell_-119.58963564767627_38.18530986960684': ['CDEC:HRS',
  'CDEC:LVT',
  'SNOTEL:575_CA_SNTL',
  'CDEC:LVM',
  'SNOTEL:771_CA_SNTL',
  'CDEC:SPS'],
 '11N_cell_-119.58849547416474_38.18527671006293': ['CDEC:HRS',
  'CDEC:LVT',
  'SNOTEL:575_CA_SNTL',
  'CDEC:LVM',
  'SNOTEL:771_CA_SNTL',
  'CDEC:SPS'],
 '11N_cell_-119.58735530252618_38.185243539472005': ['CDEC:HRS',
  

In [79]:
region = 'S_Sierras'
output_res = 100

aso_gdf = GeoSpatial(region)

#test df for now
test_gdf = aso_gdf.head(5)
test_gdf

Loading geospatial data for S_Sierras
Converting to geodataframe


,cell_id,cen_lon,cen_lat,geometry
0,11N_cell_-119.58959364631137_38.186209698720205,-119.589255,38.185854,POINT (-119.58925 38.18585)
1,11N_cell_-119.59309813700962_38.184509449472536,-119.593255,38.184854,POINT (-119.59325 38.18485)
2,11N_cell_-119.59195797185825_38.18447632413384,-119.592255,38.184854,POINT (-119.59225 38.18485)
3,11N_cell_-119.59081780857791_38.184443187748315,-119.591255,38.184854,POINT (-119.59125 38.18485)
4,11N_cell_-119.58963564767627_38.18530986960684,-119.589255,38.184854,POINT (-119.58925 38.18485)


In [87]:

# Data Access Packages
import pystac_client
import planetary_computer

# Dataframe Packages
import numpy as np
from numpy import gradient, rad2deg, arctan2
import xarray as xr
import pandas as pd

# Raster Packages
import rioxarray as rxr
from rioxarray.merge import merge_arrays
#import rasterstats as rs
from osgeo import gdal
from osgeo import gdalconst
import geopandas as gpd
import shapely
from shapely.geometry import Point, Polygon
from pyproj import CRS, Transformer

# General Packages
import os
import re
import shutil
import math
from datetime import datetime
import glob
from pprint import pprint
from typing import Union
from pathlib import Path
from tqdm import tqdm
import time
import requests
import concurrent.futures as cf #import ThreadPoolExecutor, ProcessPoolExecutor, as_completed, TimeoutError
import dask
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from tqdm import tqdm
from requests.exceptions import HTTPError
import dask_jobqueue
from dask.distributed import Client

#connecting to AWS
import warnings; warnings.filterwarnings("ignore")
import boto3
import pickle as pkl
'''
To create .netrc file:
import earthaccess
earthaccess.login(persist=True)
'''

#load access key
HOME = os.path.expanduser('~')
KEYPATH = "SWEML/AWSaccessKeys.csv"
ACCESS = pd.read_csv(f"{HOME}/{KEYPATH}")

#start session
SESSION = boto3.Session(
    aws_access_key_id=ACCESS['Access key ID'][0],
    aws_secret_access_key=ACCESS['Secret access key'][0],
)
S3 = SESSION.resource('s3')
#AWS BUCKET information
BUCKET_NAME = 'national-snow-model'
BUCKET = S3.Bucket(BUCKET_NAME)



def GeoSpatial(region):
    print(f"Loading geospatial data for {region}")
    ASO_meta_loc_DF = pd.read_csv(f"{HOME}/SWEMLv2.0/data/TrainingDFs/{region}/ASO_meta.parquet")

    cols = ['cell_id', 'x', 'y']
    ASO_meta_loc_DF = ASO_meta_loc_DF[cols]

    print(f"Converting to geodataframe")
    aso_geometry = [Point(xy) for xy in zip(ASO_meta_loc_DF['x'], ASO_meta_loc_DF['y'])]
    aso_gdf = gpd.GeoDataFrame(ASO_meta_loc_DF, geometry=aso_geometry)
    
    #renaming columns x/y to lat/long
    aso_gdf.rename(columns = {'y':'cen_lat', 'x':'cen_lon'}, inplace = True)

    return aso_gdf


def process_data_in_chunks(df, tiles, num_workers = 16):
    chunk_results = []
    
    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        futures = [executor.submit(process_single_location, (row.lat, row.lon, row.index_id, tiles)) for index, row in df.iterrows()]
        
        #for future in tqdm(as_completed(futures), total=len(futures)):
        for future in tqdm(futures):
            try:
                chunk_results.append(future.result(timeout=10))
            except TimeoutError:
                print("Processing location timed out.")
                continue
            except HTTPError as e:
                print(f"Failed to process a location due to HTTPError: {e}")
                continue  

    return pd.DataFrame(chunk_results, columns=['Elevation_m', 'Slope_Deg', 'Aspect_L'])     

#Processing using gdal
def process_single_location(args):
    #lat, lon, index_id, tiles = args
    cell_id, lat, lon, tiles = args
    #print(lat, lon, index_id, tiles)

    #signed_asset = planetary_computer.sign(tiles[int(index_id)].assets["data"])
    signed_asset = planetary_computer.sign(tiles)
    elevation = rxr.open_rasterio(signed_asset.href)
    
    slope = elevation.copy()
    aspect = elevation.copy()

    transformer = Transformer.from_crs("EPSG:4326", elevation.rio.crs, always_xy=True)
    xx, yy = transformer.transform(lon, lat)

    tilearray = np.around(elevation.values[0]).astype(int)
    geo = (math.floor(float(lon)), 90, 0.0, math.ceil(float(lat)), 0.0, -90)

    driver = gdal.GetDriverByName('MEM')
    temp_ds = driver.Create('', tilearray.shape[1], tilearray.shape[0], 1, gdalconst.GDT_Float32)

    temp_ds.GetRasterBand(1).WriteArray(tilearray)

    tilearray_np = temp_ds.GetRasterBand(1).ReadAsArray()
    grad_y, grad_x = gradient(tilearray_np)

    # Calculate slope and aspect
    slope_arr = np.sqrt(grad_x**2 + grad_y**2)
    aspect_arr = rad2deg(arctan2(-grad_y, grad_x)) % 360 
    
    slope.values[0] = slope_arr
    aspect.values[0] = aspect_arr

    elev = round(elevation.sel(x=xx, y=yy, method="nearest").values[0])
    slop = round(slope.sel(x=xx, y=yy, method="nearest").values[0])
    asp = round(aspect.sel(x=xx, y=yy, method="nearest").values[0])

    return cell_id, elev, slop, asp

def extract_terrain_data_threaded(metadata_df, max_workers=10):
    global elevation_cache 
    elevation_cache = {} 

    print('Calculating dataframe bounding box')
    bounding_box = metadata_df.geometry.total_bounds
    min_x, min_y, max_x, max_y = bounding_box[0], bounding_box[1], bounding_box[2], bounding_box[3]
    
    client = pystac_client.Client.open(
            "https://planetarycomputer.microsoft.com/api/stac/v1",
            ignore_conformance=True,
        )

    search = client.search(
                    collections=["cop-dem-glo-90"],
                    intersects = {
                            "type": "Polygon",
                            "coordinates": [[
                            [min_x, min_y],
                            [max_x, min_y],
                            [max_x, max_y],
                            [min_x, max_y],
                            [min_x, min_y]  
                        ]]})

    tiles = list(search.items())

    regions = []

    print("Retrieving Copernicus 90m DEM tiles")
    for i in tqdm(range(0, len(tiles))):
        row = [i, tiles[i].id]
        regions.append(row)
    regions = pd.DataFrame(columns = ['sliceID', 'tileID'], data = regions)
    regions = regions.set_index(regions['tileID'])
    del regions['tileID']
    print(f"There are {len(regions)} tiles in the region")

    print("Interpolating Grid Cell Spatial Features")

    # lat = metadata_df.iloc[0]['cen_lat']
    # lon = metadata_df.iloc[0]['cen_lon']
    # index_id = regions.iloc[0]['sliceID']
    # tiles = tiles
    #args = (lat, lon, index_id, tiles)


    #elev, slop, asp = process_single_location(args)

    #print(elev, slop, asp)

    #return elev, slop, asp 
    rows =[0,1,2,3,4]
    #rows = np.arange(0,len(metadata_df),1)
    # print(regions.iloc[0]['sliceID'])
    # print(tiles[int(regions.iloc[0]['sliceID'])].assets["data"])

    
    results = []
    with cf.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Start the load operations and mark each future with its process function
        jobs = {executor.submit(process_single_location, (metadata_df.iloc[i]['cell_id'], metadata_df.iloc[i]['cen_lat'], metadata_df.iloc[i]['cen_lon'], tiles[int(regions.iloc[0]['sliceID'])].assets["data"])): 
                i for i in tqdm(range(len(metadata_df)))}
        for job in cf.as_completed(jobs):
            try:
                results.append(job.result())
            except Exception as exc:
                print('%r generated an exception: %s' % (job, exc))
            else:
                print('%r page is %d bytes' % (job, len(results)))

        #     # display(results)
            # display(future)
            #results.append(future.result())
            #print(future)
    
    meta = pd.DataFrame(results, columns=['cell_id', 'Elevation_m', 'Slope_Deg', 'Aspect_Deg'])
    meta.set_index('cell_id', inplace=True)
    metadata_df.set_index('cell_id', inplace=True)
    metadata_df = pd.concat([metadata_df, meta], axis = 1)
        

    return metadata_df

In [88]:
region = 'S_Sierras'
output_res = 100

aso_gdf = GeoSpatial(region)

#test df for now
test_gdf = aso_gdf.head(20)

#needs to have output_res as inputs
metadf = extract_terrain_data_threaded(test_gdf)

# Display the results
#metadata_df.head()

Loading geospatial data for S_Sierras
Converting to geodataframe
Calculating dataframe bounding box
Retrieving Copernicus 90m DEM tiles


100%|██████████| 1/1 [00:00<00:00, 7219.11it/s]


There are 1 tiles in the region
Interpolating Grid Cell Spatial Features


100%|██████████| 20/20 [00:00<00:00, 364.47it/s]


<Future at 0x7f66d5455580 state=finished returned tuple> page is 1 bytes
<Future at 0x7f66d5455790 state=finished returned tuple> page is 2 bytes
<Future at 0x7f66c45a5430 state=finished returned tuple> page is 3 bytes
<Future at 0x7f66d54456a0 state=finished returned tuple> page is 4 bytes
<Future at 0x7f66d5438d60 state=finished returned tuple> page is 5 bytes
<Future at 0x7f66d5456070 state=finished returned tuple> page is 6 bytes
<Future at 0x7f66c4262700 state=finished returned tuple> page is 7 bytes
<Future at 0x7f66cc346160 state=finished returned tuple> page is 8 bytes
<Future at 0x7f66c45a5b20 state=finished returned tuple> page is 9 bytes
<Future at 0x7f66d54871c0 state=finished returned tuple> page is 10 bytes
<Future at 0x7f66c4262040 state=finished returned tuple> page is 11 bytes
<Future at 0x7f66b71cf160 state=finished returned tuple> page is 12 bytes
<Future at 0x7f66b71cf730 state=finished returned tuple> page is 13 bytes
<Future at 0x7f66b71cf310 state=finished return

In [89]:
metadf
    

,cen_lon,cen_lat,geometry,Elevation_m,Slope_Deg,Aspect_Deg
cell_id,,,,,,
11N_cell_-119.58959364631137_38.186209698720205,-119.589255,38.185854,POINT (-119.58925 38.18585),3168,2,18
11N_cell_-119.59309813700962_38.184509449472536,-119.593255,38.184854,POINT (-119.59325 38.18485),3156,5,342
11N_cell_-119.59195797185825_38.18447632413384,-119.592255,38.184854,POINT (-119.59225 38.18485),3158,5,34
11N_cell_-119.59081780857791_38.184443187748315,-119.591255,38.184854,POINT (-119.59125 38.18485),3165,8,32
11N_cell_-119.58963564767627_38.18530986960684,-119.589255,38.184854,POINT (-119.58925 38.18485),3167,12,90
11N_cell_-119.58849547416474_38.18527671006293,-119.588255,38.184854,POINT (-119.58825 38.18485),3166,23,125
11N_cell_-119.58735530252618_38.185243539472005,-119.587255,38.184854,POINT (-119.58725 38.18485),3140,28,138
11N_cell_-119.59428024575367_38.18364273004006,-119.594255,38.183854,POINT (-119.59425 38.18385),3152,4,63
11N_cell_-119.59314009270672_38.18360961681172,-119.593255,38.183854,POINT (-119.59325 38.18385),3152,14,94


In [74]:
meta = pd.DataFrame(meta, columns=['cell_id', 'Elevation_m', 'Slope_Deg', 'Aspect_Deg'])
meta.set_index('cell_id', inplace=True)


,cell_id,Elevation_m,Slope_Deg,Aspect_Deg
0,11N_cell_-119.58959364631137_38.186209698720205,3168,2,18
1,11N_cell_-119.59309813700962_38.184509449472536,3156,5,342
2,11N_cell_-119.58963564767627_38.18530986960684,3167,12,90
3,11N_cell_-119.59081780857791_38.184443187748315,3165,8,32
4,11N_cell_-119.59195797185825_38.18447632413384,3158,5,34


In [ ]:
# Merge with metadata
req_cols = ['cell_id', 'lat', 'lon', 'BR_Coord_Long', 'BR_Coord_Lat', 'UR_Coord_Long', 'UR_Coord_Lat',
            'UL_Coord_Long', 'UL_Coord_Lat', 'BL_Coord_Long', 'BL_Coord_Lat', 'geometry']
Result = final_df.merge(metadata[req_cols], how='left', on='cell_id')

# Column renaming and ordering
Result.rename(columns={'swe': 'ASO_SWE_in'}, inplace=True)
Result = Result[['cell_id', 'Date', 'ASO_SWE_in', 'lat', 'lon', 'nearest site 1', 'nearest site 2',
                    'nearest site 3', 'nearest site 4', 'nearest site 5', 'nearest site 6',
                    'BR_Coord_Long', 'BR_Coord_Lat', 'UR_Coord_Long', 'UR_Coord_Lat',
                    'UL_Coord_Long', 'UL_Coord_Lat', 'BL_Coord_Long', 'BL_Coord_Lat']]

# Save the merged data to a new file
output_filename = f"{HOME}/SWEML/data/NSMv2.0/data/TrainingDFs/Merged_aso_snotel_data.parquet"
Result.to_csv(output_filename, index=False)
display(Result.head(10))
print("Processed and saved data")

In [10]:
region = 'S_Sierras'
ASO_meta_loc_DF = pd.read_csv(f"{HOME}/SWEMLv2.0/data/TrainingDFs/{region}/ASO_meta.parquet")

In [6]:
#Connect nearest snotel with ASO data, this should be last for now, need to add geophysical characteristics to the site first, then this...
finaldf = GeoDF.Nearest_Snotel_2_obs(region, output_res, dropna = True) 

Snotel meta not found, retreiving from AWS S3


In [53]:
"""
A Simple implementation of parallel processing using concurrency it takes so long to execute,
Explore terrain_daskconcurrency and terrain-processing_cluster python for more optimized implementations.
"""

def process_single_location(args):
    lat, lon, regions, tiles = args
    print(lat, lon, regions, tiles)

    if (lat, lon) in elevation_cache:
        elev, slop, asp = elevation_cache[(lat, lon)]
        return elev, slop, asp

    tile_id = 'Copernicus_DSM_COG_30_N' + str(math.floor(lon)) + '_00_W' + str(math.ceil(abs(lat))) + '_00_DEM'
    index_id = regions.loc[tile_id]['sliceID']

    signed_asset = planetary_computer.sign(tiles[index_id].assets["data"])
    #print(signed_asset)
    elevation = rxr.open_rasterio(signed_asset.href)
    
    slope = elevation.copy()
    aspect = elevation.copy()

    transformer = Transformer.from_crs("EPSG:4326", elevation.rio.crs, always_xy=True)
    xx, yy = transformer.transform(lon, lat)

    tilearray = np.around(elevation.values[0]).astype(int)
    #print(tilearray)
    geo = (math.floor(float(lon)), 90, 0.0, math.ceil(float(lat)), 0.0, -90)

    no_data_value = -9999
    driver = gdal.GetDriverByName('MEM')
    temp_ds = driver.Create('', tilearray.shape[1], tilearray.shape[0], 1, gdalconst.GDT_Float32)

    temp_ds.GetRasterBand(1).WriteArray(tilearray)
    temp_ds.GetRasterBand(1).SetNoDataValue(no_data_value)
    temp_ds.SetProjection('EPSG:4326')
    temp_ds.SetGeoTransform(geo)

    tilearray_np = temp_ds.GetRasterBand(1).ReadAsArray()
    slope_arr, aspect_arr = np.gradient(tilearray_np)
    aspect_arr = np.rad2deg(np.arctan2(aspect_arr[0], aspect_arr[1]))
    
    slope.values[0] = slope_arr
    aspect.values[0] = aspect_arr

    elev = round(elevation.sel(x=xx, y=yy, method="nearest").values[0])
    slop = round(slope.sel(x=xx, y=yy, method="nearest").values[0])
    asp = round(aspect.sel(x=xx, y=yy, method="nearest").values[0])

    elevation_cache[(lat, lon)] = (elev, slop, asp)  
    return elev, slop, asp

def extract_terrain_data_threaded(metadata_df, bounding_box, max_workers=10):
    global elevation_cache 

    elevation_cache = {} 
    min_x, min_y, max_x, max_y = *bounding_box[0], *bounding_box[1]
    
    client = Client.open(
            "https://planetarycomputer.microsoft.com/api/stac/v1",
            ignore_conformance=True,
        )

    search = client.search(
                    collections=["cop-dem-glo-90"],
                    intersects = {
                            "type": "Polygon",
                            "coordinates": [[
                            [min_x, min_y],
                            [max_x, min_y],
                            [max_x, max_y],
                            [min_x, max_y],
                            [min_x, min_y]  
                        ]]})

    tiles = list(search.items())

    regions = []

    print("Retrieving Copernicus 90m DEM tiles")
    for i in tqdm(range(0, len(tiles))):
        row = [i, tiles[i].id]
        regions.append(row)
    regions = pd.DataFrame(columns = ['sliceID', 'tileID'], data = regions)
    regions = regions.set_index(regions['tileID'])
    del regions['tileID']

    print("Interpolating Grid Cell Spatial Features")

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(process_single_location, (metadata_df.iloc[i]['cen_lat'], metadata_df.iloc[i]['cen_lon'], regions, tiles))
                   for i in tqdm(range(len(metadata_df)))]
        
        results = []
        for future in tqdm(as_completed(futures), total=len(futures)):
            results.append(future.result())
    
    metadata_df['Elevation_m'], metadata_df['Slope_Deg'], metadata_df['Aspect_L'] = zip(*results)

In [ ]:
metadata_df = pd.read_csv(r"/home/vgindi/Provided_Data/Merged_aso_nearest_sites1.csv")
metadata_df= metadata_df.head(20)
bounding_box = ((-120.3763448720203, 36.29256774541929), (-118.292253412863, 38.994985247736324))    
    
extract_terrain_data_threaded(metadata_df, bounding_box)

# Display the results
metadata_df.head(10)

In [ ]:
"""
This code block crops the global coverage VIIRS data to south sierras subregion. 
"""

def crop_sierras(input_file_path, output_file_path, shapes):
    with rasterio.open(input_file_path) as src:
        out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
        out_meta = src.out_meta
        out_meta.update({"driver": "GTiff",
                         "height": out_image.shape[1],
                         "width": out_image.shape[2],
                         "transform": out_transform})
                         
        with rasterio.open(output_file_path, "w", **out_meta) as dest:
            dest.write(out_image)

def download_viirs_sca(input_dir, output_dir, shapefile_path):
    
    # Load shapes from the shapefile
    with fiona.open(shapefile_path, 'r') as shapefile:
        shapes = [feature["geometry"] for feature in shapefile]
    
    # Iterate through each year directory in the input directory
    for year_folder in os.listdir(input_dir):
        year_folder_path = os.path.join(input_dir, year_folder)
        if os.path.isdir(year_folder_path):
            # Extract year from the folder name (assuming folder names like 'WY2013')
            year = re.search(r'\d{4}', year_folder).group()
            output_year_folder = os.path.join(output_dir, year)
            os.makedirs(output_year_folder, exist_ok=True)
        
            for file_name in os.listdir(year_folder_path):        
                if file_name.endswith('.tif'):   
                    parts = file_name.split('_')
                    output_file_name = '_'.join(parts[:3]) + '.tif'
                    output_file_path = os.path.join(output_year_folder, output_file_name)
                    input_file_path = os.path.join(year_folder_path, file_name)
                    crop_sierras(input_file_path, output_file_path, shapes)
                    print(f"Processed and saved {output_file_path}")

if __name__ == "__main__":
    
    input_directory = r"/home/vgindi/VIIRS_Data"
    output_directory = r"/home/vgindi/VIIRS_Sierras"
    shapefile_path = r"/home/vgindi/Provided_Data/low_sierras_points.shp"
    download_viirs_sca(input_directory, output_directory, shapefile_path)

In [ ]:
"""
This code cell transforms the raw VIIRS tiff files to 100m resolution and saves each file in .csv format
"""
def processing_VIIRS(input_file, output_res):
    try:
        # Define the output file path for TIFFs using the original file name
        output_folder_tiff = os.path.join("/home/vgindi/Processed_VIIRS", os.path.basename(os.path.dirname(input_file)))
        os.makedirs(output_folder_tiff, exist_ok=True)
        output_file = os.path.join(output_folder_tiff, os.path.basename(input_file))

        # Reproject and resample
        ds = gdal.Open(input_file)
        if ds is None:
            print(f"Failed to open '{input_file}'. Make sure the file is a valid GeoTIFF file.")
            return None
        
        gdal.Warp(output_file, ds, dstSRS="EPSG:4326", xRes=output_res, yRes=-output_res, resampleAlg="bilinear")

        # Read the processed TIFF file using rasterio
        rds = rxr.open_rasterio(output_file)
        rds = rds.squeeze().drop("spatial_ref").drop("band")
        rds.name = "data"
        df = rds.to_dataframe().reset_index()
        return df
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

def process_and_convert_viirs(input_dir, output_res):
    # Iterate over subdirectories in the input directory
    for year in os.listdir(input_dir):
        year_dir = os.path.join(input_dir, year)
        
        if os.path.isdir(year_dir):
            for file_name in os.listdir(year_dir):
                if file_name.endswith('.tif'):
                    input_file_path = os.path.join(year_dir, file_name)
                    df = processing_VIIRS(input_file_path, output_res)
                    
                    if df is not None:
                        csv_folder = os.path.join("/home/vgindi/Processed_VIIRS", "VIIRS_csv")
                        os.makedirs(csv_folder, exist_ok=True)
                        csv_file_path = os.path.join(csv_folder, file_name.replace('.tif', '.csv'))
 
                        df.to_csv(csv_file_path, index=False)
                        print(f"Processed and saved {csv_file_path}")

if __name__ == "__main__":
    input_directory = "/home/vgindi/VIIRS_Sierras"
    output_res = 100  # Desired resolution in meters
    process_and_convert_viirs(input_directory, output_res)

In [ ]:
"""
This code cell fetches the cell id using grid_cells_meta_idx metadata for each lat/lon pair for VIIRS csv file
"""
def create_polygon(self, row):
    return Polygon([(row['BL_Coord_Long'], row['BL_Coord_Lat']),
                    (row['BR_Coord_Long'], row['BR_Coord_Lat']),
                    (row['UR_Coord_Long'], row['UR_Coord_Lat']),
                    (row['UL_Coord_Long'], row['UL_Coord_Lat'])])
    
def process_folder(self, input_folder, metadata_path, output_folder):
    # Import the metadata into a pandas DataFrame
    pred_obs_metadata_df = pd.read_csv(metadata_path)

    # Assuming create_polygon is defined elsewhere, we add a column with polygon geometries
    pred_obs_metadata_df = pred_obs_metadata_df.drop(columns=['Unnamed: 0'], axis=1)
    pred_obs_metadata_df['geometry'] = pred_obs_metadata_df.apply(self.create_polygon, axis=1)

    # Convert the DataFrame to a GeoDataFrame
    metadata = gpd.GeoDataFrame(pred_obs_metadata_df, geometry='geometry')

    # Drop coordinates columns
    metadata = metadata.drop(columns=['BL_Coord_Long', 'BL_Coord_Lat', 
                                         'BR_Coord_Long', 'BR_Coord_Lat', 
                                         'UR_Coord_Long', 'UR_Coord_Lat', 
                                         'UL_Coord_Long', 'UL_Coord_Lat'], axis=1)

    # List all CSV files in the input folder
    csv_files = [f for f in os.listdir(input_folder) if f.endswith('.csv')]

    for csv_file in csv_files:
        input_path = os.path.join(input_folder, csv_file)
        output_path = os.path.join(output_folder, csv_file)

        # Check if the output file already exists
        if os.path.exists(output_path):
            print(f"CSV file {csv_file} already exists in the output folder.")
            continue

        # Process each CSV file
        viirs_sca_df = pd.read_csv(input_path)

        # Convert the "aso_swe_df" into a GeoDataFrame with point geometries
        geometry = [Point(xy) for xy in zip(viirs_sca_df['x'], viirs_sca_df['y'])]
        viirs_sca_geo = gpd.GeoDataFrame(viirs_sca_df, geometry=geometry)
        result = gpd.sjoin(viirs_sca_geo, metadata, how='left', predicate='within', op = 'intersects')

        # Select specific columns for the final DataFrame
        Final_df = result[['y', 'x', 'data', 'cell_id']]
        Final_df.rename(columns={'data': 'VIIRS_SCA'}, inplace=True)

        # Drop rows where 'cell_id' is NaN
        if Final_df['cell_id'].isnull().values.any():
            Final_df = Final_df.dropna(subset=['cell_id'])

        # Save the processed DataFrame to a CSV file
        Final_df.to_csv(output_path, index=False)
        print(f"Processed {csv_file}")

if __name__ == "__main__":
    input_folder = r""
    metadata_path = r""
    output_folder = r""
    process_folder(input_folder, metadata_path, output_folder)